# Slicing CDR Relation Extraction 

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import sys
sys.path.append('/dfs/scratch0/vschen/metal')

import metal
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [3]:
print('PyTorch: ', torch.__version__)
print('MeTaL:   ', metal.__version__)
print('Python:  ', sys.version)
print('Python:  ', sys.version_info)

PyTorch:  1.0.0
MeTaL:    0.3.3
Python:   3.6.7 (default, Dec  8 2018, 17:35:14) 
[GCC 5.4.0 20160609]
Python:   sys.version_info(major=3, minor=6, micro=7, releaselevel='final', serial=0)


## Initalize CDR Dataset
To uncompress the SQLite db: ```bzip2 -d cdr.db.bz2```

In [4]:
from metal.contrib.backends.wrapper import SnorkelDataset
import os

db_conn_str   = os.path.join(os.getcwd(),"cdr.db")
candidate_def = ['ChemicalDisease', ['chemical', 'disease']]

train, dev, test = SnorkelDataset.splits(db_conn_str, 
                                         candidate_def, 
                                         max_seq_len=125)

print(f'[TRAIN] {len(train)}')
print(f'[DEV]   {len(dev)}')
print(f'[TEST]  {len(test)}')

Connected to sqlite:////dfs/scratch0/vschen/metal/metal/contrib/slicing/CDR/cdr.db
Connected to sqlite:////dfs/scratch0/vschen/metal/metal/contrib/slicing/CDR/cdr.db
Connected to sqlite:////dfs/scratch0/vschen/metal/metal/contrib/slicing/CDR/cdr.db
[TRAIN] 8272
[DEV]   888
[TEST]  4620


## Get Pretrained Embeddings

Download [GloVe embeddings](http://nlp.stanford.edu/data/glove.6B.zip):
`wget http://nlp.stanford.edu/data/glove.6B.zip \
&& mkdir -p glove.6B \
&& unzip glove.6B.zip -d glove.6B \
&& rm glove.6B.zip`

In [5]:
from embeddings import EmbeddingLoader, load_embeddings
emb_path  = "glove.6B/glove.6B.50d.txt"
embs  = EmbeddingLoader(emb_path, fmt='text')

## (a) `Oracle`: EndModel Trained on Full GT

In [6]:
from metal.end_model import EndModel
from metal.modules import LSTMModule
use_cuda = torch.cuda.is_available()

wembs = load_embeddings(train.word_dict, embs)
lstm = LSTMModule(embed_size=50, 
                  hidden_size=100, 
                  embeddings=wembs,
                  lstm_reduction='attention', 
                  dropout=0, 
                  num_layers=1, 
                  freeze=False)

oracle = EndModel([200, 2], input_module=lstm, seed=123, use_cuda=use_cuda)
oracle.config['train_config']['optimizer_config']['optimizer_common']['lr'] = 0.01
oracle.config['train_config']['validation_metric'] = 'f1'
oracle.config['train_config']['batch_size'] = 32
oracle.config['train_config']['n_epochs'] = 10

%time oracle.train_model(train, dev_data=dev)
oracle.score(test, metric=['precision', 'recall', 'f1'])

Loaded 77.0% (7656/9946) pretrained embeddings
Using pretrained embeddings.
Embeddings shape = (9946, 50)
The embeddings are NOT FROZEN
Using lstm_reduction = 'attention'

Network architecture:
Sequential(
  (0): Sequential(
    (0): LSTMModule(
      (embeddings): Embedding(9946, 50)
      (lstm): LSTM(50, 100, batch_first=True, bidirectional=True)
    )
    (1): ReLU()
  )
  (1): Linear(in_features=200, out_features=2, bias=True)
)

Using GPU...



Saving model at iteration 0 with best score 0.602
[E:0]	Train Loss: 0.550	Dev f1: 0.602



[E:1]	Train Loss: 0.296	Dev f1: 0.586



[E:2]	Train Loss: 0.166	Dev f1: 0.588



Saving model at iteration 3 with best score 0.604
[E:3]	Train Loss: 0.111	Dev f1: 0.604


...


[E:6]	Train Loss: 0.051	Dev f1: 0.586



[E:7]	Train Loss: 0.045	Dev f1: 0.599



[E:8]	Train Loss: 0.044	Dev f1: 0.584



[E:9]	Train Loss: 0.037	Dev f1: 0.599
Restoring best model from iteration 5 with score 0.610
Finished Training
F1: 0.610
        y=1    y=2   
 l=1    182    119   
 l=2    114    473   
CPU times: user 14min 58s, sys: 14.7 s, total: 15min 12s
Wall time: 15min 9s
Precision: 0.522
Recall: 0.625
F1: 0.569
        y=1    y=2   
 l=1    941    860   
 l=2    564   2255   


[0.5224875069405885, 0.6252491694352159, 0.5692679975801573]

## (b) `BaseWeak`: EndModel trained on weak labels

In [8]:
from metal.contrib.slicing.CDR.wrapper \
    import SnorkelDataset as SnorkelSliceDataset

train_weak = SnorkelSliceDataset(
    db_conn_str,
    candidate_def,
    split=0,
    use_lfs=1
)

# dev_weak = SnorkelSliceDataset(
#     db_conn_str,
#     candidate_def,
#     split=1,
#     use_lfs=1
# )

Connected to sqlite:////dfs/scratch0/vschen/metal/metal/contrib/slicing/CDR/cdr.db


In [9]:
from metal.end_model import EndModel
from metal.modules import LSTMModule
use_cuda = torch.cuda.is_available()

wembs = load_embeddings(train.word_dict, embs)
lstm = LSTMModule(embed_size=50, 
                  hidden_size=100,
                  embeddings=wembs,
                  lstm_reduction='attention', 
                  dropout=0,
                  num_layers=1,
                  freeze=False)

base_weak = EndModel([200, 2], input_module=lstm, seed=123, use_cuda=use_cuda)

base_weak.config['train_config']['optimizer_config']['optimizer_common']['lr'] = 0.01
base_weak.config['train_config']['validation_metric'] = 'f1'
base_weak.config['train_config']['batch_size'] = 32
base_weak.config['train_config']['n_epochs'] = 10

%time base_weak.train_model(train_weak, dev_data=dev)
# %time base_weak.train_model(dev_weak, dev_data=dev)
base_weak_scores = base_weak.score(test, metric=['precision', 'recall', 'f1'])

Loaded 77.0% (7656/9946) pretrained embeddings
Using pretrained embeddings.
Embeddings shape = (9946, 50)
The embeddings are NOT FROZEN
Using lstm_reduction = 'attention'

Network architecture:
Sequential(
  (0): Sequential(
    (0): LSTMModule(
      (embeddings): Embedding(9946, 50)
      (lstm): LSTM(50, 100, batch_first=True, bidirectional=True)
    )
    (1): ReLU()
  )
  (1): Linear(in_features=200, out_features=2, bias=True)
)

Using GPU...



Saving model at iteration 0 with best score 0.505
[E:0]	Train Loss: 0.621	Dev f1: 0.505



Saving model at iteration 1 with best score 0.506
[E:1]	Train Loss: 0.593	Dev f1: 0.506



Saving model at iteration 2 with best score 0.518
[E:2]	Train Loss: 0.582	Dev f1: 0.518



[E:3]	Train Loss: 0.578	Dev f1: 0.511


...


[E:5]	Train Loss: 0.574	Dev f1: 0.504



[E:6]	Train Loss: 0.573	Dev f1: 0.508



Saving model at iteration 7 with best score 0.519
[E:7]	Train Loss: 0.572	Dev f1: 0.519



[E:8]	Train Loss: 0.572	Dev f1: 0.511



[E:9]	Train Loss: 0.571	Dev f1: 0.502
Restoring best model from iteration 7 with score 0.519
Finished Training
F1: 0.519
        y=1    y=2   
 l=1    265    461   
 l=2    31     131   
CPU times: user 13min 53s, sys: 16.6 s, total: 14min 10s
Wall time: 14min 6s
Precision: 0.334
Recall: 0.868
F1: 0.483
        y=1    y=2   
 l=1   1306   2602   
 l=2    199    513   


## Generate `L_*` to target slices

In [10]:
from labeling_functions import LFs
print ([lf.__name__ for lf in LFs])

['LF_c_cause_d', 'LF_c_d', 'LF_c_induced_d', 'LF_c_treat_d', 'LF_c_treat_d_wide', 'LF_closer_chem', 'LF_closer_dis', 'LF_ctd_marker_c_d', 'LF_ctd_marker_induce', 'LF_ctd_therapy_treat', 'LF_ctd_unspecified_treat', 'LF_ctd_unspecified_induce', 'LF_d_following_c', 'LF_d_induced_by_c', 'LF_d_induced_by_c_tight', 'LF_d_treat_c', 'LF_develop_d_following_c', 'LF_far_c_d', 'LF_far_d_c', 'LF_improve_before_disease', 'LF_in_ctd_therapy', 'LF_in_ctd_marker', 'LF_in_patient_with', 'LF_induce', 'LF_induce_name', 'LF_induced_other', 'LF_level', 'LF_measure', 'LF_neg_d', 'LF_risk_d', 'LF_treat_d', 'LF_uncertain', 'LF_weak_assertions']


In [15]:
%%time 
from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.annotations import LabelAnnotator
labeler = LabelAnnotator(lfs=LFs)
L_train = labeler.apply(split=0)
L_dev = labeler.apply(split=1) # used for debugging
L_test = labeler.apply(split=2) # used for evaluation

from snorkel.learning.structure import DependencySelector
ds = DependencySelector()
deps = ds.select(L_train, threshold=0.1)
from snorkel.learning import GenerativeModel

# need to extract `accs` from gen_model
gen_model = GenerativeModel(lf_propensity=True)
gen_model.train(
    L_train, deps=deps, decay=0.95, step_size=0.1/L_train.shape[0], reg_param=0.0
)

accs = np.array(gen_model.learned_lf_stats()['Accuracy'])
accs[np.isnan(accs)] = 0
accs = np.minimum(accs, 0.999)

# train_marginals = gen_model.marginals(L_train)

Clearing existing...


  0%|          | 6/8272 [00:00<02:24, 57.24it/s]

Running UDF...


100%|██████████| 8272/8272 [00:37<00:00, 218.01it/s]


Clearing existing...


...

Running UDF...


  0%|          | 0/4620 [00:00<?, ?it/s]

Clearing existing...
Running UDF...


100%|██████████| 4620/4620 [00:21<00:00, 214.99it/s]


Inferred cardinality: 2
CPU times: user 2min 41s, sys: 752 ms, total: 2min 42s
Wall time: 2min 44s


### Custom Slicing Dataset

In [12]:
# passes L values as supervision
train_slice = SnorkelSliceDataset(
    db_conn_str,
    candidate_def,
    split=0,
    L_train=L_train,
)

# contains weak labels as priors 
train_slice_weak = SnorkelSliceDataset(
    db_conn_str,
    candidate_def,
    split=0,
    L_train=L_train,
    use_lfs=1
)

# # for debugging purposes (overfitting to dev set)
# dev_slice = SnorkelSliceDataset(
#     db_conn_str,
#     candidate_def,
#     split=1,
#     L_train=L_dev,
# )

# dev_slice_weak = SnorkelSliceDataset(
#     db_conn_str,
#     candidate_def,
#     split=1,
#     L_train=L_train,
#     use_lfs=1
# )

Connected to sqlite:////dfs/scratch0/vschen/metal/metal/contrib/slicing/CDR/cdr.db
Connected to sqlite:////dfs/scratch0/vschen/metal/metal/contrib/slicing/CDR/cdr.db


## (c) `SliceUW`: Unweighted SliceModel with `rw=False`

In [13]:
from metal.contrib.slicing.online_dp import SliceDPModel, LinearModule

In [16]:
wembs = load_embeddings(train.word_dict, embs)
lstm = LSTMModule(embed_size=50, 
                  hidden_size=100, 
                  embeddings=wembs,
                  lstm_reduction='attention', 
                  dropout=0, 
                  num_layers=1, 
                  freeze=False)

r_dim = 200
rw = False
slice_uw = SliceDPModel(lstm, accs, r_dim, rw, seed=123, use_cuda=True)

slice_uw.config['train_config']['optimizer_config']['optimizer_common']['lr'] = 0.01
slice_uw.config['train_config']['validation_metric'] = 'f1'
slice_uw.config['train_config']['batch_size'] = 32
slice_uw.config['train_config']['n_epochs'] = 10

%time slice_uw.train_model(train_slice, dev_data=dev)
slice_uw_scores = slice_uw.score(test, metric=['precision', 'recall', 'f1'])

Loaded 77.0% (7656/9946) pretrained embeddings
Using pretrained embeddings.
Embeddings shape = (9946, 50)
The embeddings are NOT FROZEN
Using lstm_reduction = 'attention'
Slice Heads:
Input Network: Sequential(
  (0): Sequential(
    (0): LSTMModule(
      (embeddings): Embedding(9946, 50)
      (lstm): LSTM(50, 100, batch_first=True, bidirectional=True)
    )
    (1): ReLU()
  )
)
L_head: Linear(in_features=200, out_features=33, bias=False)
Y_head: Linear(in_features=200, out_features=2, bias=False)
Using GPU...


/dfs/scratch0/vschen/snorkel/snorkel/learning/gen_learning.py:350: RuntimeWarning: invalid value encountered in double_scalars
  "Precision": tp / (tp + fp),
/dfs/scratch0/vschen/snorkel/snorkel/learning/gen_learning.py:352: RuntimeWarning: invalid value encountered in double_scalars
  "Accuracy": (tp + tn) / coverage,
/dfs/scratch0/vschen/snorkel-pytorch/venv/lib/python3.6/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
/dfs/scratch0/vschen/metal/metal/contrib/slicing/online_dp.py:103: RuntimeWarning: divide by zero encountered in log
  self.w = torch.from_numpy(np.log(accs / (1-accs))).float()


/dfs/scratch0/vschen/snorkel-pytorch/venv/lib/python3.6/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")



Saving model at iteration 0 with best score 0.528
[E:0]	Train Loss: 0.445	Dev f1: 0.528



Saving model at iteration 1 with best score 0.602
[E:1]	Train Loss: 0.430	Dev f1: 0.602


...


[E:5]	Train Loss: 0.416	Dev f1: 0.596



[E:6]	Train Loss: 0.414	Dev f1: 0.580



[E:7]	Train Loss: 0.413	Dev f1: 0.584



[E:8]	Train Loss: 0.412	Dev f1: 0.601



[E:9]	Train Loss: 0.412	Dev f1: 0.587
Restoring best model from iteration 1 with score 0.602
Finished Training
F1: 0.602
        y=1    y=2   
 l=1    259    306   
 l=2    37     286   
CPU times: user 25min 54s, sys: 28.2 s, total: 26min 22s
Wall time: 26min 18s
Precision: 0.417
Recall: 0.863
F1: 0.563
        y=1    y=2   
 l=1   1299   1814   
 l=2    206   1301   


## (d) `SliceOurs`: Attention SliceModel with `rw=True`

In [17]:
wembs = load_embeddings(train.word_dict, embs)
lstm = LSTMModule(embed_size=50, 
                  hidden_size=100, 
                  embeddings=wembs,
                  lstm_reduction='attention', 
                  dropout=0, 
                  num_layers=1, 
                  freeze=False)


r_dim = 200
rw = True
slice_ours = SliceDPModel(lstm, accs, r_dim, rw, seed=123, use_cuda=True)

slice_ours.config['train_config']['optimizer_config']['optimizer_common']['lr'] = 0.01
slice_ours.config['train_config']['validation_metric'] = 'f1'
slice_ours.config['train_config']['batch_size'] = 32
slice_ours.config['train_config']['n_epochs'] = 10

%time slice_ours.train_model(train_slice, dev_data=dev)
slice_ours_scores = slice_ours.score(test, metric=['precision', 'recall', 'f1'])

Loaded 77.0% (7656/9946) pretrained embeddings
Using pretrained embeddings.
Embeddings shape = (9946, 50)
The embeddings are NOT FROZEN
Using lstm_reduction = 'attention'
Slice Heads:
Input Network: Sequential(
  (0): Sequential(
    (0): LSTMModule(
      (embeddings): Embedding(9946, 50)
      (lstm): LSTM(50, 100, batch_first=True, bidirectional=True)
    )
    (1): ReLU()
  )
)
L_head: Linear(in_features=200, out_features=33, bias=False)
Y_head: Linear(in_features=400, out_features=2, bias=False)
Using GPU...


/dfs/scratch0/vschen/metal/metal/contrib/slicing/online_dp.py:170: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  A = F.softmax(self.forward_L(x)).unsqueeze(1)



Saving model at iteration 0 with best score 0.561
[E:0]	Train Loss: 0.444	Dev f1: 0.561



Saving model at iteration 1 with best score 0.575
[E:1]	Train Loss: 0.430	Dev f1: 0.575



[E:2]	Train Loss: 0.422	Dev f1: 0.563


...


[E:5]	Train Loss: 0.413	Dev f1: 0.584



[E:6]	Train Loss: 0.411	Dev f1: 0.594



[E:7]	Train Loss: 0.410	Dev f1: 0.581



[E:8]	Train Loss: 0.409	Dev f1: 0.594



[E:9]	Train Loss: 0.409	Dev f1: 0.573
Restoring best model from iteration 3 with score 0.598
Finished Training
F1: 0.598
        y=1    y=2   
 l=1    254    299   
 l=2    42     293   
CPU times: user 36min 56s, sys: 42.2 s, total: 37min 38s
Wall time: 37min 34s
Precision: 0.412
Recall: 0.863
F1: 0.558
        y=1    y=2   
 l=1   1299   1854   
 l=2    206   1261   


## (e) `SliceOursWeak`: Slice Model with $\tilde{Y}$ priors

In [18]:
from metal.contrib.slicing.online_dp import SliceDPModel, LinearModule
from metal.modules import LSTMModule

wembs = load_embeddings(train.word_dict, embs)
lstm = LSTMModule(embed_size=50, 
                  hidden_size=100, 
                  embeddings=wembs,
                  lstm_reduction='attention', 
                  dropout=0, 
                  num_layers=1, 
                  freeze=False)

r_dim = 200
rw = True
slice_ours_weak = SliceDPModel(lstm, accs, r_dim, rw, seed=123, use_cuda=True)

slice_ours_weak.config['train_config']['optimizer_config']['optimizer_common']['lr'] = 0.01
slice_ours_weak.config['train_config']['validation_metric'] = 'f1'
slice_ours_weak.config['train_config']['batch_size'] = 32
slice_ours_weak.config['train_config']['n_epochs'] = 10

%time slice_ours_weak.train_model(train_slice_weak, dev_data=dev)
slice_ours_weak_scores = slice_ours_weak.score(test, metric=['precision', 'recall', 'f1'])

Loaded 77.0% (7656/9946) pretrained embeddings
Using pretrained embeddings.
Embeddings shape = (9946, 50)
The embeddings are NOT FROZEN
Using lstm_reduction = 'attention'
Slice Heads:
Input Network: Sequential(
  (0): Sequential(
    (0): LSTMModule(
      (embeddings): Embedding(9946, 50)
      (lstm): LSTM(50, 100, batch_first=True, bidirectional=True)
    )
    (1): ReLU()
  )
)
L_head: Linear(in_features=200, out_features=33, bias=False)
Y_head: Linear(in_features=400, out_features=2, bias=False)
Using GPU...



Saving model at iteration 0 with best score 0.503
[E:0]	Train Loss: 0.448	Dev f1: 0.503



Saving model at iteration 1 with best score 0.517
[E:1]	Train Loss: 0.438	Dev f1: 0.517



[E:2]	Train Loss: 0.432	Dev f1: 0.506



[E:3]	Train Loss: 0.429	Dev f1: 0.509


...


[E:5]	Train Loss: 0.426	Dev f1: 0.506



[E:6]	Train Loss: 0.426	Dev f1: 0.495



[E:7]	Train Loss: 0.425	Dev f1: 0.504



[E:8]	Train Loss: 0.424	Dev f1: 0.504



[E:9]	Train Loss: 0.424	Dev f1: 0.507
Restoring best model from iteration 4 with score 0.522
Finished Training
F1: 0.522
        y=1    y=2   
 l=1    274    480   
 l=2    22     112   
CPU times: user 36min 4s, sys: 41 s, total: 36min 45s
Wall time: 36min 39s
Precision: 0.334
Recall: 0.906
F1: 0.488
        y=1    y=2   
 l=1   1364   2723   
 l=2    141    392   


## (f) `SliceUWWeak`: Unweighted Slice model with $\tilde{Y}$ priors

In [19]:
from metal.contrib.slicing.online_dp import SliceDPModel, LinearModule
from metal.modules import LSTMModule

wembs = load_embeddings(train.word_dict, embs)
lstm = LSTMModule(embed_size=50, 
                  hidden_size=100, 
                  embeddings=wembs,
                  lstm_reduction='attention', 
                  dropout=0, 
                  num_layers=1, 
                  freeze=False)


r_dim = 200
rw = False
slice_uw_weak = SliceDPModel(lstm, accs, r_dim, rw, seed=123, use_cuda=True)

slice_uw_weak.config['train_config']['optimizer_config']['optimizer_common']['lr'] = 0.01
slice_uw_weak.config['train_config']['validation_metric'] = 'f1'
slice_uw_weak.config['train_config']['batch_size'] = 32
slice_uw_weak.config['train_config']['n_epochs'] = 10

%time slice_uw_weak.train_model(train_slice_weak, dev_data=dev)
slice_uw_weak_scores = slice_uw_weak.score(test, metric=['precision', 'recall', 'f1'])


Loaded 77.0% (7656/9946) pretrained embeddings
Using pretrained embeddings.
Embeddings shape = (9946, 50)
The embeddings are NOT FROZEN
Using lstm_reduction = 'attention'
Slice Heads:
Input Network: Sequential(
  (0): Sequential(
    (0): LSTMModule(
      (embeddings): Embedding(9946, 50)
      (lstm): LSTM(50, 100, batch_first=True, bidirectional=True)
    )
    (1): ReLU()
  )
)
L_head: Linear(in_features=200, out_features=33, bias=False)
Y_head: Linear(in_features=200, out_features=2, bias=False)
Using GPU...



Saving model at iteration 0 with best score 0.510
[E:0]	Train Loss: 0.448	Dev f1: 0.510



Saving model at iteration 1 with best score 0.513
[E:1]	Train Loss: 0.438	Dev f1: 0.513



[E:2]	Train Loss: 0.432	Dev f1: 0.508



[E:3]	Train Loss: 0.429	Dev f1: 0.505


...


Saving model at iteration 5 with best score 0.516
[E:5]	Train Loss: 0.426	Dev f1: 0.516



[E:6]	Train Loss: 0.425	Dev f1: 0.506



[E:7]	Train Loss: 0.425	Dev f1: 0.508



[E:8]	Train Loss: 0.424	Dev f1: 0.511



[E:9]	Train Loss: 0.424	Dev f1: 0.513
Restoring best model from iteration 5 with score 0.516
Finished Training
F1: 0.516
        y=1    y=2   
 l=1    268    475   
 l=2    28     117   
CPU times: user 25min 51s, sys: 28.9 s, total: 26min 20s
Wall time: 26min 15s
Precision: 0.331
Recall: 0.851
F1: 0.477
        y=1    y=2   
 l=1   1281   2587   
 l=2    224    528   


## Slice-specific scores

In [20]:
# TODO: don't call private fns
Yp_oracle, Y = oracle._get_predictions(test)
Yp_base_weak, Y = base_weak._get_predictions(test)
Yp_slice_uw, Y = slice_uw._get_predictions(test)
Yp_slice_ours, Y = slice_ours._get_predictions(test)
Yp_slice_ours_weak, Y = slice_ours_weak._get_predictions(test)
Yp_slice_uw_weak, Y = slice_uw_weak._get_predictions(test)

#### `slice_ours` (re-weighting, accuracy priors) vs. `base_weak` (end_model trained on weak labels)

In [55]:
from metal.contrib.slicing.experiment_utils import compare_LF_slices
compare_LF_slices(Yp_slice_ours, Yp_base_weak, 
                  Y, L_test, LFs, metric='accuracy', delta_threshold=0.05)

[LF_c_treat_d] delta: 0.6298, OURS: 0.8383, BASE: 0.2085
[LF_c_treat_d_wide] delta: 0.4787, OURS: 0.7775, BASE: 0.2989
[LF_closer_chem] delta: 0.3050, OURS: 0.5553, BASE: 0.2503
[LF_closer_dis] delta: 0.1039, OURS: 0.4545, BASE: 0.3506
[LF_ctd_therapy_treat] delta: 0.6574, OURS: 0.8406, BASE: 0.1833
[LF_ctd_unspecified_treat] delta: 0.5571, OURS: 0.7820, BASE: 0.2249
[LF_d_following_c] delta: 0.5000, OURS: 1.0000, BASE: 0.5000
[LF_d_treat_c] delta: 0.0970, OURS: 0.4606, BASE: 0.3636
[LF_far_c_d] delta: 0.2724, OURS: 0.6586, BASE: 0.3862
[LF_far_d_c] delta: 0.1288, OURS: 0.5613, BASE: 0.4325
[LF_improve_before_disease] delta: 0.1111, OURS: 0.4444, BASE: 0.3333
[LF_in_ctd_therapy] delta: 0.2763, OURS: 0.5684, BASE: 0.2921
[LF_in_ctd_marker] delta: 0.0738, OURS: 0.6142, BASE: 0.5403
[LF_in_patient_with] delta: 0.2222, OURS: 0.3333, BASE: 0.1111
[LF_induce] delta: 0.0792, OURS: 0.6238, BASE: 0.5446
[LF_induced_other] delta: 0.3333, OURS: 0.6215, BASE: 0.2881
[LF_level] delta: 0.5526, OURS:

#### `slice_ours_weak` (slice model with weak priors + reweighting) vs. `base_weak` (end_model trained on weak labels)

In [54]:
compare_LF_slices(Yp_slice_ours_weak, Yp_base_weak,
                  Y, L_test, LFs, metric='accuracy', delta_threshold=0.05)

[LF_d_following_c] delta: 0.5000, OURS: 1.0000, BASE: 0.5000
improved 1/33


#### `slice_ours_weak` vs. `oracle` (trained on full GT)

In [53]:
compare_LF_slices(Yp_slice_ours, Yp_oracle,
                  Y, L_test, LFs, metric='accuracy', delta_threshold=0.05)

[LF_ctd_marker_c_d] delta: 0.0511, OURS: 0.7500, BASE: 0.6989
[LF_ctd_marker_induce] delta: 0.0951, OURS: 0.8016, BASE: 0.7065
[LF_ctd_therapy_treat] delta: 0.0876, OURS: 0.8406, BASE: 0.7530
[LF_ctd_unspecified_induce] delta: 0.0920, OURS: 0.8006, BASE: 0.7086
[LF_d_induced_by_c_tight] delta: 0.0510, OURS: 0.6939, BASE: 0.6429
[LF_develop_d_following_c] delta: 0.2500, OURS: 1.0000, BASE: 0.7500
[LF_level] delta: 0.1579, OURS: 0.7632, BASE: 0.6053
improved 7/33


#### `slice_ours` vs. `Yp_slice_uw` (unweighted slice model)

In [52]:
compare_LF_slices(Yp_slice_ours, Yp_slice_uw,
                  Y, L_test, LFs, metric='accuracy', delta_threshold=0.05)

[LF_c_treat_d] delta: 0.1447, OURS: 0.8383, BASE: 0.6936
[LF_c_treat_d_wide] delta: 0.0921, OURS: 0.7775, BASE: 0.6854
[LF_ctd_therapy_treat] delta: 0.1434, OURS: 0.8406, BASE: 0.6972
[LF_ctd_unspecified_treat] delta: 0.1003, OURS: 0.7820, BASE: 0.6817
[LF_induced_other] delta: 0.0621, OURS: 0.6215, BASE: 0.5593
[LF_level] delta: 0.2632, OURS: 0.7632, BASE: 0.5000
[LF_measure] delta: 0.0909, OURS: 0.7273, BASE: 0.6364
[LF_neg_d] delta: 0.1250, OURS: 0.4464, BASE: 0.3214
[LF_risk_d] delta: 0.0714, OURS: 0.4286, BASE: 0.3571
improved 9/33
